In [19]:
import pandas as pd
import numpy as np

In [20]:
#export_cn 파일 불러오기
export_cn=pd.read_table('Trade_Map_-_List_of_products_exported_by_China.txt', sep='\t')
export_cn.rename(columns={'Code':'HS'}, inplace=True)
export_cn.drop(export_cn.index[0], inplace=True)
export_cn['HS']=export_cn['HS'].astype('int64')

In [21]:
#HS17 코드 불러와서 export_cn 파일과 병합
hs_bec=pd.read_csv('hs_bec17.csv')
export_cn=pd.merge(export_cn, hs_bec, how='left')

In [22]:
#최근 10년간 거래가 없는 품목 제외시킴
no_amount_cn=export_cn[(export_cn['Exported value in 2012']==0)
                       & (export_cn['Exported value in 2013']==0) 
                       & (export_cn['Exported value in 2014']==0) 
                       & (export_cn['Exported value in 2015']==0) 
                       & (export_cn['Exported value in 2016']==0) 
                       & (export_cn['Exported value in 2017']==0)  
                       & (export_cn['Exported value in 2018']==0) 
                       & (export_cn['Exported value in 2019']==0) 
                       & (export_cn['Exported value in 2020']==0) 
                       & (export_cn['Exported value in 2021']==0)].index
export_cn.drop(no_amount_cn, inplace=True)

In [23]:
#HS 999999품목은 따로 묶기 위해 임의의 BEC 번호 999999를 만들어 줌
export_cn.loc[export_cn['HS'] == 999999, 'BEC17'] = 999999

In [24]:
#BEC17 결측치 확인
export_cn[export_cn['BEC17'].isnull()]

,HS,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12,BEC17
4983,30626,"Cold-water shrimps and prawns """"Pandalus spp.,...",7744,505,227,124,127,0,0,0,0,0,NaN,NaN
5013,382490,Chemical products and preparations of the chem...,2419825,2409610,2727016,2775854,2586928,409,0,33,0,0,NaN,NaN
5020,310310,Superphosphates (excluding those in tablets or...,631437,419992,469765,499361,318399,0,0,0,0,0,NaN,NaN
5023,380850,Goods of heading 3808 containing one or more o...,3880,3213,4847,4150,3961,0,0,0,0,0,NaN,NaN
5026,293500,Sulphonamides,572491,625633,623120,606418,664590,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,401163,"Pneumatic tyres, new, of rubber, having a """"he...",173498,128085,110730,75642,73226,0,0,0,0,0,NaN,NaN
6033,401193,"Pneumatic tyres, new, of rubber, of a kind use...",116359,113061,146266,133655,129594,133,0,0,0,0,NaN,NaN
6034,401199,"Pneumatic tyres, new, of rubber (excluding hav...",335490,321121,303951,270009,256963,311,0,0,0,0,NaN,NaN
6064,600531,Unbleached or bleached warp knit fabrics of sy...,116494,139465,172153,170903,143868,0,0,0,0,0,NaN,NaN


In [25]:
#BEC17 결측치 있는 행들만 모아 데이터 프레임을 만듬(아래에서 BEC12 파일과 병합을 위해)
BEC17_nan_cn=export_cn[export_cn['BEC17'].isnull()]

In [26]:
#BEC12 파일과 BEC17 결측치가 담긴 데이터프레임을 병합
hs_bec12=pd.read_csv('hs_bec12.csv')
BEC17_nan_cn=pd.merge(BEC17_nan_cn, hs_bec12, how='left')

In [27]:
#BEC12와 모두 매치 되어 결측치 없음을 확인
BEC17_nan_cn.isnull().sum()

HS                         0
Product label              0
Exported value in 2012     0
Exported value in 2013     0
Exported value in 2014     0
Exported value in 2015     0
Exported value in 2016     0
Exported value in 2017     0
Exported value in 2018     0
Exported value in 2019     0
Exported value in 2020     0
Exported value in 2021     0
Unnamed: 12               78
BEC17                     78
BEC12                      1
dtype: int64

In [28]:
BEC17_nan_cn[BEC17_nan_cn['BEC12'].isnull()]

,HS,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12,BEC17,BEC12
37,854800,"Electrical parts of machinery or apparatus, n....",0,0,0,0,0,0,12,0,0,0,NaN,NaN,NaN


In [29]:
#BEC17로 채워진 행들(export_cn)과 BEC12(BEC17_nan_cn)로 채워진 행들을 합치기
export_cn_complete=pd.merge(export_cn, BEC17_nan_cn, how='left')

In [30]:
#컬럼들을 보기 좋게 배열하기 위해 컬럼명 확인
export_cn_complete.columns

Index(['HS', 'Product label', 'Exported value in 2012',
       'Exported value in 2013', 'Exported value in 2014',
       'Exported value in 2015', 'Exported value in 2016',
       'Exported value in 2017', 'Exported value in 2018',
       'Exported value in 2019', 'Exported value in 2020',
       'Exported value in 2021', 'Unnamed: 12', 'BEC17', 'BEC12'],
      dtype='object')

In [31]:
export_cn_complete=export_cn_complete[['HS', 'BEC17', 'BEC12', 'Product label', 'Exported value in 2012',
       'Exported value in 2013', 'Exported value in 2014',
       'Exported value in 2015', 'Exported value in 2016',
       'Exported value in 2017', 'Exported value in 2018',
       'Exported value in 2019', 'Exported value in 2020',
       'Exported value in 2021', 'Unnamed: 12']]


In [32]:
export_cn_complete

,HS,BEC17,BEC12,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12
0,851712,613202.0,NaN,"Telephones for cellular networks """"mobile tele...",81453710,95625511,115918827,124901982,116148676,127265887,141683901,125432114,126086675,127908542,NaN
1,847130,612020.0,NaN,"Data-processing machines, automatic, portable,...",113784253,110802220,108658457,90457283,79408394,89749076,95883926,95669671,115314593,124188992,NaN
2,854232,611220.0,NaN,Electronic integrated circuits as memories,14374599,17692084,17566232,21566607,23241587,29621004,43966741,52446486,55864236,68017077,NaN
3,851770,611220.0,NaN,"Parts of telephone sets, telephones for cellul...",39594274,47116398,45105066,49585071,48463570,50776282,51951119,47859965,45200289,49192067,NaN
4,851762,612020.0,NaN,"Machines for the reception, conversion and tra...",26543598,26879247,28640165,32437464,31987803,36155599,42267557,46600996,48185653,45936899,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5232,810930,3111.0,NaN,Zirconium waste and scrap (excluding ash and r...,12,81,167,207,0,52,155,0,0,0,NaN
5233,811222,3111.0,NaN,Chromium waste and scrap (excluding ash and re...,0,0,0,2,0,0,26,50,0,0,NaN
5234,600531,NaN,411220.0,Unbleached or bleached warp knit fabrics of sy...,116494,139465,172153,170903,143868,0,0,0,0,0,NaN
5235,870370,513202.0,NaN,Motor cars and other motor vehicles principall...,0,0,0,0,0,37,0,15,0,0,NaN


In [33]:
#BEC17 컬럼과 BEC12 컬럼을 합쳐주기 위해 결측치를 0으로 처리
export_cn_complete['BEC17']=export_cn_complete['BEC17'].fillna(0)
export_cn_complete['BEC12']=export_cn_complete['BEC12'].fillna(0)
#BEC17 컬럼과 BEC12 컬럼의 값을 더해서 BEC 컬럼을 만들고 기존의 BEC17 컬럼과 BEC12 컬럼은 제외
export_cn_complete['BEC']=export_cn_complete['BEC17']+export_cn_complete['BEC12']
export_cn_complete=export_cn_complete.drop(columns=['BEC17','BEC12'])

In [34]:
export_cn_complete=export_cn_complete[['HS', 'BEC','Product label', 'Exported value in 2012',
       'Exported value in 2013', 'Exported value in 2014',
       'Exported value in 2015', 'Exported value in 2016',
       'Exported value in 2017', 'Exported value in 2018',
       'Exported value in 2019', 'Exported value in 2020',
       'Exported value in 2021', 'Unnamed: 12']]

In [35]:
export_cn_complete

,HS,BEC,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12
0,851712,613202.0,"Telephones for cellular networks """"mobile tele...",81453710,95625511,115918827,124901982,116148676,127265887,141683901,125432114,126086675,127908542,NaN
1,847130,612020.0,"Data-processing machines, automatic, portable,...",113784253,110802220,108658457,90457283,79408394,89749076,95883926,95669671,115314593,124188992,NaN
2,854232,611220.0,Electronic integrated circuits as memories,14374599,17692084,17566232,21566607,23241587,29621004,43966741,52446486,55864236,68017077,NaN
3,851770,611220.0,"Parts of telephone sets, telephones for cellul...",39594274,47116398,45105066,49585071,48463570,50776282,51951119,47859965,45200289,49192067,NaN
4,851762,612020.0,"Machines for the reception, conversion and tra...",26543598,26879247,28640165,32437464,31987803,36155599,42267557,46600996,48185653,45936899,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5232,810930,3111.0,Zirconium waste and scrap (excluding ash and r...,12,81,167,207,0,52,155,0,0,0,NaN
5233,811222,3111.0,Chromium waste and scrap (excluding ash and re...,0,0,0,2,0,0,26,50,0,0,NaN
5234,600531,411220.0,Unbleached or bleached warp knit fabrics of sy...,116494,139465,172153,170903,143868,0,0,0,0,0,NaN
5235,870370,513202.0,Motor cars and other motor vehicles principall...,0,0,0,0,0,37,0,15,0,0,NaN


In [36]:
export_cn_complete.to_csv("export_cn.csv", index = False)